In [6]:
import os
import sys
import cv2
import time
import selenium
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tensorflow.keras.preprocessing.image import img_to_array
from imutils.object_detection import non_max_suppression
from imageai.Detection import ObjectDetection

In [4]:
driver_path = "/home/ashwinkumar/Projects/Driver/chromedriver_linux64/chromedriver"

In [83]:
wd = get_google_images_page(driver_path, "pikachu")

In [36]:
tabs = wd.find_elements_by_css_selector("a[jslog='11106']")

In [26]:
print(wd.find_elements_by_css_selector("a[jslog='11106'] span.hIOe2")[9].text)
wd.find_elements_by_css_selector("a[jslog='11106'] span.hIOe2")[9].location

wallpaper


{'x': 1326, 'y': 159}

In [2]:
# fetch the window height and scroll to the end to load more images
def scroll_to_end(wd):
    height = wd.execute_script("return document.body.scrollHeight")
    wd.execute_script("window.scrollTo(0, {})".format(height))

# go to google images page with the query
def get_google_images_page(driver_path, query):
    wd = webdriver.Chrome(executable_path=driver_path)
    wd.get("http://www.google.com/")
    wd.maximize_window()
#     print(wd.current_url)
    
    # select images button
    wd.find_element_by_css_selector("a.gb_g[data-pid='2']").click()
    
    # find the search box, clear it and type the query passed
    search = wd.find_element_by_css_selector("input[title='Search']")
    search.clear()
    search.send_keys(query)
    search.send_keys(Keys.RETURN)
    time.sleep(1)
    
    return wd
     
# scrape images from google
def scrap_images(wd, max_count=50, num_scrolls=3): 
    
    image_urls_batch = set()
    counter = 0
    width = wd.get_window_size()["width"]
    
    while len(image_urls_batch) < max_count:
        
        try:
            print("********{}**********".format(counter))
            if counter == 0:

                for j in range(num_scrolls):

                    print("----------{}-{}----------".format(counter, j))

                    if wd.find_element_by_css_selector("div.OuJzKb.Yu2Dnd").is_displayed():
                        break

                    image_elements = wd.find_elements_by_css_selector("img.rg_i.Q4LuWd")

                    for element in image_elements:
                        url = element.get_attribute("src")
                        try:
                            image_content = requests.get(url).content
                            image_urls_batch.add(url)

                        except Exception as e:
                            continue 
                    scroll_to_end(wd)
                    print("image_urls_batch:", len(image_urls_batch))

                    if wd.find_element_by_css_selector("input.mye4qd").is_displayed():
                        wd.find_element_by_css_selector("input.mye4qd").click()

                    if len(image_urls_batch) > max_count:
                        break

                wd.execute_script("window.scroll(0, 0)")

            else:

                tab_name = wd.find_elements_by_css_selector("a[jslog='11106']")[counter - 1].text
                print(tab_name)

                tab = wd.find_elements_by_css_selector("a[jslog='11106']")[counter - 1]
                if counter >= 8:
                    wd.find_element_by_css_selector("div.cEW58.khjlM").click()
                    wd.implicitly_wait(1)

                tab.click()
                wd.implicitly_wait(1)

                for j in range(num_scrolls):

                    print("##########{}#{}##########".format(counter, j))

                    if wd.find_element_by_css_selector("div.OuJzKb.Yu2Dnd").is_displayed():
                        break
                    image_elements = wd.find_elements_by_css_selector("img.rg_i.Q4LuWd")

                    for element in image_elements:
                        url = element.get_attribute("src")
                        try:
                            image_content = requests.get(url).content
                            image_urls_batch.add(url)

                        except Exception as e:
                            continue 

                    scroll_to_end(wd)
                    print("image_urls_batch:", len(image_urls_batch))

                    if wd.find_element_by_css_selector("input.mye4qd").is_displayed():
                        wd.find_element_by_css_selector("input.mye4qd").click()

                    if len(image_urls_batch) > max_count:
                        break

                wd.back()                
                wd.implicitly_wait(1)

            counter += 1
        
        except Exception as e:
            break
            
    return list(image_urls_batch)

# function to save images to the desired folder
def save_images_to_folder(query, image_urls, folder_path, extension):
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    for i in range(len(image_urls)):
        
        file_name = "{}_{}.{}".format(query, i, extension)
        file_name = os.path.join(folder_path, file_name)
        image = requests.get(image_urls[i]).content
        image = Image.open(BytesIO(image)).convert("RGB")
        image.save(file_name)

In [5]:
# generate pokemon names list from website
wd = webdriver.Chrome(executable_path=driver_path)
wd.get("https://www.pokemon.com/us/pokedex/")
print(wd.current_url)
wd.implicitly_wait(1)
num_characters = 30
pokemon_names = []

# handle cookies
try:
    wd.find_element_by_css_selector("button.onetrust-close-btn-handler.onetrust-close-btn-ui.banner-close-button.ot-close-icon").click()
    
except Exception as e:
    pass

wd.find_element_by_css_selector("a#loadMore").click()

while len(pokemon_names) < num_characters:
    scroll_to_end(wd)
    names = [i.text for i in wd.find_elements_by_css_selector("h5")]
    
    names = list(set(names).difference(set(pokemon_names)))
    pokemon_names.extend(names)
wd.quit()

https://www.pokemon.com/us/pokedex/


In [6]:
len(pokemon_names)

36

In [107]:
pokemon_names[4]

'Ivysaur'

In [110]:
# scrape images for each pokemon name and save them in a folder
for name in pokemon_names[4:]:
    image_urls = []
    max_count = 1000
    num_scrolls = 4
    
    folder_path = "/home/ashwinkumar/Projects/Cartoon_Generator/images/{}".format(name)
    wd = get_google_images_page(driver_path, name)

    while len(image_urls) < max_count:
        image_urls_batch = scrap_images(wd, max_count, num_scrolls)
        image_urls.extend(image_urls_batch)

    save_images_to_folder(name, image_urls, folder_path, extension="jpg")
    print("{} done".format(name))
    wd.quit()

********0**********
----------0-0----------
image_urls_batch: 6
----------0-1----------
image_urls_batch: 58
----------0-2----------
image_urls_batch: 80
----------0-3----------
image_urls_batch: 139
********1**********
bulbasaur
##########1#0##########
image_urls_batch: 143
##########1#1##########
image_urls_batch: 189
##########1#2##########
image_urls_batch: 213
##########1#3##########
image_urls_batch: 248
********2**********
pokemon
##########2#0##########
image_urls_batch: 250
##########2#1##########
image_urls_batch: 285
##########2#2##########
image_urls_batch: 302
##########2#3##########
image_urls_batch: 357
********3**********
super smash bros
##########3#0##########
image_urls_batch: 361
##########3#1##########
image_urls_batch: 409
##########3#2##########
image_urls_batch: 437
##########3#3##########
image_urls_batch: 488
********4**********
shiny
##########4#0##########
image_urls_batch: 493
##########4#1##########
image_urls_batch: 539
##########4#2##########
image_urls_

image_urls_batch: 498
********4**********
pokemon
##########4#0##########
image_urls_batch: 504
##########4#1##########
image_urls_batch: 534
##########4#2##########
image_urls_batch: 550
##########4#3##########
image_urls_batch: 586
********5**********
sunglasses
##########5#0##########
image_urls_batch: 590
##########5#1##########
image_urls_batch: 636
##########5#2##########
image_urls_batch: 666
##########5#3##########
image_urls_batch: 710
********6**********
pikachu
##########6#0##########
image_urls_batch: 712
##########6#1##########
image_urls_batch: 751
##########6#2##########
image_urls_batch: 778
##########6#3##########
image_urls_batch: 821
********7**********
evolution
##########7#0##########
image_urls_batch: 825
##########7#1##########
image_urls_batch: 872
##########7#2##########
image_urls_batch: 896
##########7#3##########
image_urls_batch: 942
********8**********
cute
##########8#0##########
image_urls_batch: 949
##########8#1##########
image_urls_batch: 984
########

image_urls_batch: 894
********8**********
gen 1
##########8#0##########
image_urls_batch: 896
##########8#1##########
image_urls_batch: 934
##########8#2##########
image_urls_batch: 947
##########8#3##########
image_urls_batch: 994
********9**********
anthro
##########9#0##########
image_urls_batch: 997
##########9#1##########
image_urls_batch: 997
##########9#2##########
image_urls_batch: 997
##########9#3##########
image_urls_batch: 997
********10**********
base set
##########10#0##########
image_urls_batch: 1011
Kakuna done
********0**********
----------0-0----------
image_urls_batch: 7
----------0-1----------
image_urls_batch: 58
----------0-2----------
image_urls_batch: 80
----------0-3----------
image_urls_batch: 141
********1**********
mega
##########1#0##########
image_urls_batch: 144
##########1#1##########
image_urls_batch: 179
##########1#2##########
image_urls_batch: 197
##########1#3##########
image_urls_batch: 244
********2**********
pokemon
##########2#0##########
image_

image_urls_batch: 1030
Weedle done
********0**********
----------0-0----------
image_urls_batch: 5
----------0-1----------
image_urls_batch: 57
----------0-2----------
image_urls_batch: 80
----------0-3----------
image_urls_batch: 128
********1**********
alolan
##########1#0##########
image_urls_batch: 131
##########1#1##########
image_urls_batch: 170
##########1#2##########
image_urls_batch: 197
##########1#3##########
image_urls_batch: 258
********2**********
rattata
##########2#0##########
image_urls_batch: 262
##########2#1##########
image_urls_batch: 303
##########2#2##########
image_urls_batch: 330
##########2#3##########
image_urls_batch: 379
********3**********
shiny
##########3#0##########
image_urls_batch: 383
##########3#1##########
image_urls_batch: 433
##########3#2##########
image_urls_batch: 452
##########3#3##########
image_urls_batch: 502
********4**********
pokemon
##########4#0##########
image_urls_batch: 506
##########4#1##########
image_urls_batch: 523
##########4#

image_urls_batch: 311
##########2#2##########
image_urls_batch: 338
##########2#3##########
image_urls_batch: 397
********3**********
pokemon
##########3#0##########
image_urls_batch: 402
##########3#1##########
image_urls_batch: 431
##########3#2##########
image_urls_batch: 442
##########3#3##########
image_urls_batch: 478
********4**********
shiny
##########4#0##########
image_urls_batch: 484
##########4#1##########
image_urls_batch: 524
##########4#2##########
image_urls_batch: 545
##########4#3##########
image_urls_batch: 599
********5**********
real
##########5#0##########
image_urls_batch: 602
##########5#1##########
image_urls_batch: 643
##########5#2##########
image_urls_batch: 663
##########5#3##########
image_urls_batch: 706
********6**********
ash
##########6#0##########
image_urls_batch: 711
##########6#1##########
image_urls_batch: 755
##########6#2##########
image_urls_batch: 777
##########6#3##########
image_urls_batch: 836
********7**********
magikarp
##########7#0#####

image_urls_batch: 624
********6**********
purple
##########6#0##########
image_urls_batch: 628
##########6#1##########
image_urls_batch: 646
##########6#2##########
image_urls_batch: 655
##########6#3##########
image_urls_batch: 690
********7**********
nidoking
##########7#0##########
image_urls_batch: 693
##########7#1##########
image_urls_batch: 710
##########7#2##########
image_urls_batch: 724
##########7#3##########
image_urls_batch: 765
********8**********
nidorina
##########8#0##########
image_urls_batch: 767
##########8#1##########
image_urls_batch: 774
##########8#2##########
image_urls_batch: 779
##########8#3##########
image_urls_batch: 802
********9**********
evolve
##########9#0##########
image_urls_batch: 802
##########9#1##########
image_urls_batch: 812
##########9#2##########
image_urls_batch: 818
##########9#3##########
image_urls_batch: 840
********10**********
tower defense
##########10#0##########
image_urls_batch: 842
##########10#1##########
image_urls_batch: 862
#

image_urls_batch: 905
********7**********
cartoon
##########7#0##########
image_urls_batch: 913
##########7#1##########
image_urls_batch: 953
##########7#2##########
image_urls_batch: 983
##########7#3##########
image_urls_batch: 1044
Pikachu done
********0**********
----------0-0----------
image_urls_batch: 5
----------0-1----------
image_urls_batch: 57
----------0-2----------
image_urls_batch: 80
----------0-3----------
image_urls_batch: 133
********1**********
evolution
##########1#0##########
image_urls_batch: 136
##########1#1##########
image_urls_batch: 177
##########1#2##########
image_urls_batch: 201
##########1#3##########
image_urls_batch: 244
********2**********
gengar
##########2#0##########
image_urls_batch: 245
##########2#1##########
image_urls_batch: 285
##########2#2##########
image_urls_batch: 306
##########2#3##########
image_urls_batch: 362
********3**********
art
##########3#0##########
image_urls_batch: 367
##########3#1##########
image_urls_batch: 412
##########3

image_urls_batch: 1001
Nidoran♀ done
********0**********
----------0-0----------
image_urls_batch: 8
----------0-1----------
image_urls_batch: 49
----------0-2----------
image_urls_batch: 80
----------0-3----------
image_urls_batch: 139
********1**********
evolution
##########1#0##########
image_urls_batch: 144
##########1#1##########
image_urls_batch: 186
##########1#2##########
image_urls_batch: 213
##########1#3##########
image_urls_batch: 274
********2**********
anthro
##########2#0##########
image_urls_batch: 282
##########2#1##########
image_urls_batch: 338
##########2#2##########
image_urls_batch: 354
##########2#3##########
********3**********
female
##########3#0##########
image_urls_batch: 359
##########3#1##########
image_urls_batch: 396
##########3#2##########
image_urls_batch: 410
##########3#3##########
image_urls_batch: 478
********4**********
shiny
##########4#0##########
image_urls_batch: 482
##########4#1##########
image_urls_batch: 520
##########4#2##########
image_u

image_urls_batch: 296
##########2#2##########
image_urls_batch: 317
##########2#3##########
image_urls_batch: 369
********3**********
evolved
##########3#0##########
image_urls_batch: 373
##########3#1##########
image_urls_batch: 410
##########3#2##########
image_urls_batch: 435
##########3#3##########
image_urls_batch: 492
********4**********
real life
##########4#0##########
image_urls_batch: 496
##########4#1##########
image_urls_batch: 538
##########4#2##########
image_urls_batch: 568
##########4#3##########
image_urls_batch: 614
********5**********
pokemon
##########5#0##########
image_urls_batch: 617
##########5#1##########
image_urls_batch: 641
##########5#2##########
image_urls_batch: 652
##########5#3##########
image_urls_batch: 697
********6**********
shiny
##########6#0##########
image_urls_batch: 703
##########6#1##########
image_urls_batch: 753
##########6#2##########
image_urls_batch: 769
##########6#3##########
image_urls_batch: 819
********7**********
pangolin
#########

In [15]:
# contour detection in saved images

images_folder = "/home/ashwinkumar/Projects/Cartoon_Generator/images"

for folder in os.listdir(images_folder):

    if not os.path.exists(os.path.join(images_folder, folder, "roi")):
        os.makedirs(os.path.join(images_folder, folder, "roi"))

    for file in os.listdir(os.path.join(images_folder, folder)):
        if file != "roi":
            print(file)
            image = cv2.imread(os.path.join(images_folder, folder, file))
            print(image.shape)
            image = cv2.GaussianBlur(image, (5, 5), 0)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 225, 255, cv2.THRESH_BINARY_INV)[1]
        #     cv2.imshow("thresh", thresh)
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()

            # find contours
            contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
            bboxes = [cv2.boundingRect(c) for c in contours]
            
            try:
                _, sorted_contours = zip(*sorted(zip(bboxes, contours), key=lambda x: x[0][0]))

                # sorted_contours = [i for i in sorted_contours if len(i) > 4]
            #     copy = cv2.drawContours(image.copy(), sorted_contours, 0, (0, 0, 255), 2)

                bbox = cv2.boundingRect(sorted_contours[0])
                x, y, w, h = bbox
                roi = image[y: y + h, x: x + w]
                new = cv2.imwrite(os.path.join(images_folder, folder, "roi", file), roi)
                
            #     cv2.rectangle(copy, bbox, (0, 0, 255), 2)
            #     cv2.imshow("copy", copy)
            #     cv2.waitKey(0)
            #     cv2.destroyAllWindows()
            
            except Exception as e:
                continue

Venusaur_317.jpg
(168, 299, 3)
Venusaur_571.jpg
(267, 189, 3)
Venusaur_528.jpg
(225, 225, 3)
Venusaur_594.jpg
(225, 225, 3)
Venusaur_504.jpg
(225, 225, 3)
Venusaur_297.jpg
(225, 225, 3)
Venusaur_982.jpg
(184, 273, 3)
Venusaur_636.jpg
(243, 208, 3)
Venusaur_566.jpg
(189, 267, 3)
Venusaur_719.jpg
(225, 225, 3)
Venusaur_59.jpg
(202, 250, 3)
Venusaur_844.jpg
(226, 223, 3)
Venusaur_949.jpg
(169, 297, 3)
Venusaur_638.jpg
(244, 207, 3)
Venusaur_172.jpg
(191, 263, 3)
Venusaur_22.jpg
(225, 225, 3)
Venusaur_236.jpg
(203, 248, 3)
Venusaur_222.jpg
(175, 287, 3)
Venusaur_426.jpg
(194, 259, 3)
Venusaur_229.jpg
(168, 300, 3)
Venusaur_848.jpg
(217, 232, 3)
Venusaur_300.jpg
(168, 300, 3)
Venusaur_641.jpg
(265, 190, 3)
Venusaur_323.jpg
(225, 225, 3)
Venusaur_272.jpg
(174, 290, 3)
Venusaur_279.jpg
(300, 168, 3)
Venusaur_559.jpg
(194, 259, 3)
Venusaur_856.jpg
(225, 225, 3)
Venusaur_474.jpg
(225, 225, 3)
Venusaur_776.jpg
(128, 128, 3)
Venusaur_461.jpg
(194, 259, 3)
Venusaur_482.jpg
(156, 322, 3)
Venusaur_4

Venusaur_322.jpg
(225, 225, 3)
Venusaur_976.jpg
(225, 225, 3)
Venusaur_193.jpg
(225, 225, 3)
Venusaur_156.jpg
(318, 159, 3)
Venusaur_0.jpg
(225, 225, 3)
Venusaur_627.jpg
(212, 238, 3)
Venusaur_541.jpg
(228, 221, 3)
Venusaur_283.jpg
(170, 297, 3)
Venusaur_149.jpg
(225, 225, 3)
Venusaur_204.jpg
(168, 300, 3)
Venusaur_550.jpg
(170, 297, 3)
Venusaur_577.jpg
(177, 284, 3)
Venusaur_629.jpg
(168, 300, 3)
Venusaur_208.jpg
(194, 259, 3)
Venusaur_205.jpg
(225, 225, 3)
Venusaur_618.jpg
(225, 225, 3)
Venusaur_65.jpg
(210, 210, 3)
Venusaur_11.jpg
(207, 243, 3)
Venusaur_553.jpg
(202, 250, 3)
Venusaur_923.jpg
(182, 276, 3)
Venusaur_537.jpg
(220, 229, 3)
Venusaur_398.jpg
(210, 240, 3)
Venusaur_498.jpg
(205, 246, 3)
Venusaur_119.jpg
(300, 168, 3)
Venusaur_756.jpg
(194, 259, 3)
Venusaur_660.jpg
(168, 300, 3)
Venusaur_798.jpg
(225, 225, 3)
Venusaur_294.jpg
(205, 246, 3)
Venusaur_715.jpg
(168, 299, 3)
Venusaur_184.jpg
(184, 274, 3)
Venusaur_158.jpg
(183, 275, 3)
Venusaur_137.jpg
(159, 318, 3)
Venusaur_644

(168, 300, 3)
Venusaur_927.jpg
(177, 284, 3)
Venusaur_752.jpg
(318, 159, 3)
Venusaur_262.jpg
(228, 221, 3)
Venusaur_628.jpg
(225, 225, 3)
Venusaur_51.jpg
(168, 300, 3)
Venusaur_404.jpg
(212, 238, 3)
Venusaur_28.jpg
(163, 309, 3)
Venusaur_30.jpg
(194, 259, 3)
Venusaur_850.jpg
(177, 284, 3)
Venusaur_334.jpg
(254, 198, 3)
Venusaur_876.jpg
(224, 225, 3)
Venusaur_940.jpg
(225, 225, 3)
Venusaur_360.jpg
(168, 300, 3)
Venusaur_395.jpg
(162, 310, 3)
Venusaur_298.jpg
(200, 252, 3)
Venusaur_819.jpg
(203, 248, 3)
Venusaur_433.jpg
(188, 268, 3)
Venusaur_78.jpg
(225, 225, 3)
Venusaur_958.jpg
(117, 429, 3)
Venusaur_198.jpg
(237, 213, 3)
Venusaur_922.jpg
(189, 267, 3)
Venusaur_525.jpg
(318, 159, 3)
Venusaur_373.jpg
(162, 311, 3)
Venusaur_785.jpg
(225, 225, 3)
Venusaur_4.jpg
(263, 192, 3)
Venusaur_746.jpg
(159, 317, 3)
Venusaur_278.jpg
(183, 275, 3)
Venusaur_312.jpg
(168, 300, 3)
Venusaur_58.jpg
(265, 190, 3)
Venusaur_637.jpg
(171, 294, 3)
Venusaur_477.jpg
(267, 189, 3)
Venusaur_634.jpg
(257, 196, 3)
V

(212, 238, 3)
Venusaur_428.jpg
(202, 250, 3)
Venusaur_647.jpg
(225, 225, 3)
Venusaur_94.jpg
(243, 207, 3)
Venusaur_524.jpg
(300, 168, 3)
Venusaur_107.jpg
(190, 265, 3)
Venusaur_780.jpg
(225, 225, 3)
Venusaur_466.jpg
(192, 216, 3)
Venusaur_842.jpg
(178, 283, 3)
Venusaur_171.jpg
(225, 225, 3)
Venusaur_12.jpg
(163, 310, 3)
Venusaur_260.jpg
(165, 305, 3)
Venusaur_663.jpg
(231, 218, 3)
Venusaur_192.jpg
(201, 251, 3)
Venusaur_34.jpg
(199, 253, 3)
Venusaur_84.jpg
(258, 195, 3)
Venusaur_350.jpg
(260, 194, 3)
Venusaur_69.jpg
(265, 190, 3)
Venusaur_356.jpg
(194, 259, 3)
Venusaur_500.jpg
(168, 300, 3)
Venusaur_761.jpg
(259, 194, 3)
Venusaur_480.jpg
(168, 299, 3)
Venusaur_1002.jpg
(220, 220, 3)
Venusaur_832.jpg
(194, 259, 3)
Venusaur_912.jpg
(217, 232, 3)
Venusaur_653.jpg
(225, 225, 3)
Venusaur_267.jpg
(159, 318, 3)
Venusaur_676.jpg
(225, 225, 3)
Venusaur_95.jpg
(177, 284, 3)
Venusaur_271.jpg
(128, 128, 3)
Venusaur_349.jpg
(159, 318, 3)
Venusaur_337.jpg
(225, 225, 3)
Venusaur_470.jpg
(173, 292, 3)

(225, 225, 3)
Sandslash_543.jpg
(225, 225, 3)
Sandslash_525.jpg
(225, 225, 3)
Sandslash_624.jpg
(221, 228, 3)
Sandslash_611.jpg
(259, 194, 3)
Sandslash_270.jpg
(164, 307, 3)
Sandslash_912.jpg
(259, 194, 3)
Sandslash_588.jpg
(195, 258, 3)
Sandslash_449.jpg
(160, 315, 3)
Sandslash_727.jpg
(159, 318, 3)
Sandslash_448.jpg
(201, 251, 3)
Sandslash_777.jpg
(225, 225, 3)
Sandslash_888.jpg
(168, 300, 3)
Sandslash_108.jpg
(194, 259, 3)
Sandslash_385.jpg
(249, 203, 3)
Sandslash_11.jpg
(255, 197, 3)
Sandslash_178.jpg
(194, 259, 3)
Sandslash_117.jpg
(215, 234, 3)
Sandslash_20.jpg
(155, 324, 3)
Sandslash_787.jpg
(209, 189, 3)
Sandslash_174.jpg
(256, 197, 3)
Sandslash_756.jpg
(192, 263, 3)
Sandslash_932.jpg
(224, 224, 3)
Sandslash_169.jpg
(259, 194, 3)
Sandslash_491.jpg
(184, 273, 3)
Sandslash_937.jpg
(168, 300, 3)
Sandslash_429.jpg
(265, 190, 3)
Sandslash_132.jpg
(152, 332, 3)
Sandslash_981.jpg
(233, 216, 3)
Sandslash_204.jpg
(216, 233, 3)
Sandslash_84.jpg
(194, 259, 3)
Sandslash_587.jpg
(259, 194, 

...

Sandshrew_232.jpg
(194, 259, 3)
Sandshrew_636.jpg
(183, 276, 3)
Sandshrew_220.jpg
(194, 259, 3)
Sandshrew_213.jpg
(260, 194, 3)
Sandshrew_797.jpg
(169, 299, 3)
Sandshrew_223.jpg
(216, 234, 3)
Sandshrew_424.jpg
(168, 300, 3)
Sandshrew_281.jpg
(194, 259, 3)
Sandshrew_21.jpg
(192, 262, 3)
Sandshrew_936.jpg
(220, 220, 3)
Sandshrew_340.jpg
(225, 225, 3)
Sandshrew_807.jpg
(212, 238, 3)
Sandshrew_1001.jpg
(267, 189, 3)
Sandshrew_609.jpg
(251, 201, 3)
Sandshrew_644.jpg
(194, 259, 3)
Sandshrew_840.jpg
(183, 275, 3)
Sandshrew_555.jpg
(225, 225, 3)
Sandshrew_53.jpg
(179, 282, 3)
Sandshrew_425.jpg
(184, 273, 3)
Sandshrew_529.jpg
(280, 180, 3)
Sandshrew_570.jpg
(225, 225, 3)
Sandshrew_881.jpg
(167, 301, 3)
Sandshrew_1006.jpg
(225, 225, 3)
Sandshrew_427.jpg
(243, 208, 3)
Sandshrew_505.jpg
(242, 208, 3)
Sandshrew_124.jpg
(183, 275, 3)
Sandshrew_891.jpg
(249, 202, 3)
Sandshrew_103.jpg
(180, 279, 3)
Sandshrew_855.jpg
(183, 275, 3)
Sandshrew_461.jpg
(225, 225, 3)
Sandshrew_615.jpg
(225, 225, 3)
Sandshre

(224, 224, 3)
Caterpie_776.jpg
(166, 304, 3)
Caterpie_724.jpg
(205, 246, 3)
Caterpie_611.jpg
(159, 318, 3)
Caterpie_1021.jpg
(168, 300, 3)
Caterpie_125.jpg
(223, 226, 3)
Caterpie_278.jpg
(224, 225, 3)
Caterpie_794.jpg
(194, 259, 3)
Caterpie_437.jpg
(225, 225, 3)
Caterpie_513.jpg
(177, 284, 3)
Caterpie_314.jpg
(225, 225, 3)
Caterpie_51.jpg
(168, 300, 3)
Caterpie_543.jpg
(275, 183, 3)
Caterpie_215.jpg
(177, 284, 3)
Caterpie_353.jpg
(300, 168, 3)
Caterpie_46.jpg
(225, 225, 3)
Caterpie_1040.jpg
(225, 225, 3)
Caterpie_299.jpg
(322, 156, 3)
Caterpie_406.jpg
(217, 232, 3)
Caterpie_951.jpg
(220, 229, 3)
Caterpie_595.jpg
(222, 227, 3)
Caterpie_929.jpg
(168, 300, 3)
Caterpie_828.jpg
(225, 225, 3)
Caterpie_438.jpg
(168, 300, 3)
Caterpie_648.jpg
(228, 221, 3)
Caterpie_721.jpg
(153, 329, 3)
Caterpie_962.jpg
(248, 203, 3)
Caterpie_180.jpg
(232, 217, 3)
Caterpie_359.jpg
(225, 225, 3)
Caterpie_576.jpg
(238, 212, 3)
Caterpie_592.jpg
(265, 190, 3)
Caterpie_377.jpg
(225, 225, 3)
Caterpie_767.jpg
(225, 22

(300, 168, 3)
Caterpie_335.jpg
(194, 259, 3)
Caterpie_761.jpg
(225, 225, 3)
Caterpie_297.jpg
(225, 225, 3)
Caterpie_763.jpg
(259, 194, 3)
Caterpie_921.jpg
(225, 225, 3)
Caterpie_893.jpg
(189, 267, 3)
Caterpie_183.jpg
(217, 232, 3)
Caterpie_661.jpg
(168, 300, 3)
Caterpie_649.jpg
(194, 259, 3)
Caterpie_216.jpg
(238, 211, 3)
Caterpie_881.jpg
(194, 259, 3)
Caterpie_19.jpg
(235, 215, 3)
Caterpie_684.jpg
(263, 191, 3)
Caterpie_682.jpg
(177, 284, 3)
Caterpie_783.jpg
(168, 300, 3)
Caterpie_598.jpg
(157, 321, 3)
Caterpie_194.jpg
(213, 236, 3)
Caterpie_298.jpg
(142, 354, 3)
Caterpie_456.jpg
(174, 290, 3)
Caterpie_842.jpg
(225, 225, 3)
Caterpie_79.jpg
(266, 190, 3)
Caterpie_664.jpg
(216, 234, 3)
Caterpie_768.jpg
(259, 194, 3)
Caterpie_948.jpg
(159, 318, 3)
Caterpie_453.jpg
(230, 220, 3)
Caterpie_788.jpg
(168, 300, 3)
Caterpie_151.jpg
(194, 259, 3)
Caterpie_1008.jpg
(225, 225, 3)
Caterpie_534.jpg
(168, 300, 3)
Caterpie_810.jpg
(259, 194, 3)
Caterpie_610.jpg
(173, 290, 3)
Caterpie_803.jpg
(225, 225

(263, 192, 3)
Caterpie_612.jpg
(225, 225, 3)
Caterpie_844.jpg
(259, 194, 3)
Caterpie_514.jpg
(148, 179, 3)
Caterpie_479.jpg
(180, 280, 3)
Caterpie_864.jpg
(275, 183, 3)
Caterpie_746.jpg
(235, 215, 3)
Caterpie_634.jpg
(128, 128, 3)
Caterpie_736.jpg
(225, 225, 3)
Caterpie_104.jpg
(194, 259, 3)
Caterpie_953.jpg
(163, 310, 3)
Caterpie_427.jpg
(209, 241, 3)
Caterpie_394.jpg
(194, 259, 3)
Caterpie_720.jpg
(194, 259, 3)
Caterpie_382.jpg
(225, 225, 3)
Caterpie_791.jpg
(265, 190, 3)
Caterpie_139.jpg
(225, 225, 3)
Caterpie_309.jpg
(235, 214, 3)
Caterpie_871.jpg
(225, 225, 3)
Caterpie_969.jpg
(205, 245, 3)
Caterpie_937.jpg
(259, 194, 3)
Caterpie_781.jpg
(168, 300, 3)
Caterpie_496.jpg
(168, 300, 3)
Caterpie_1015.jpg
(197, 256, 3)
Caterpie_789.jpg
(227, 222, 3)
Caterpie_631.jpg
(194, 259, 3)
Caterpie_972.jpg
(266, 189, 3)
Caterpie_775.jpg
(189, 267, 3)
Caterpie_484.jpg
(201, 251, 3)
Caterpie_87.jpg
(204, 200, 3)
Caterpie_599.jpg
(176, 286, 3)
Caterpie_800.jpg
(267, 189, 3)
Caterpie_348.jpg
(243, 20

Caterpie_996.jpg
(314, 160, 3)
Caterpie_659.jpg
(212, 238, 3)
Caterpie_15.jpg
(168, 300, 3)
Caterpie_257.jpg
(229, 220, 3)
Caterpie_129.jpg
(257, 196, 3)
Caterpie_857.jpg
(142, 355, 3)
Caterpie_1014.jpg
(194, 259, 3)
Caterpie_494.jpg
(225, 224, 3)
Caterpie_303.jpg
(259, 194, 3)
Caterpie_167.jpg
(196, 257, 3)
Caterpie_344.jpg
(216, 233, 3)
Caterpie_548.jpg
(200, 200, 3)
Caterpie_797.jpg
(134, 375, 3)
Caterpie_848.jpg
(168, 300, 3)
Caterpie_285.jpg
(168, 299, 3)
Caterpie_939.jpg
(225, 225, 3)
Caterpie_717.jpg
(238, 211, 3)
Caterpie_526.jpg
(225, 225, 3)
Caterpie_72.jpg
(194, 259, 3)
Caterpie_281.jpg
(235, 215, 3)
Caterpie_9.jpg
(238, 212, 3)
Caterpie_1024.jpg
(300, 168, 3)
Caterpie_1005.jpg
(265, 190, 3)
Caterpie_731.jpg
(186, 271, 3)
Caterpie_757.jpg
(168, 300, 3)
Caterpie_686.jpg
(153, 330, 3)
Caterpie_369.jpg
(225, 225, 3)
Caterpie_635.jpg
(225, 224, 3)
Caterpie_792.jpg
(199, 253, 3)
Caterpie_127.jpg
(169, 299, 3)
Caterpie_1012.jpg
(265, 190, 3)
Caterpie_338.jpg
(342, 147, 3)
Caterpie

In [61]:
f = "/home/ashwinkumar/Projects/Cartoon_Generator/images/Ivysaur/Ivysaur_0.jpg"
# f = "/home/ashwinkumar/Downloads/person.jpeg"

# i = Image.open(f).convert("L")
# i = img_to_array(i)
j = cv2.imread(f)
copy = j.copy()
# cv2.imshow("copy", j)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("/home/ashwinkumar/Projects/models/resnet50_coco_best_v2.1.0.h5")
detector.loadModel()

In [62]:
detections = detector.detectObjectsFromImage(input_image=f, output_image_path="/home/ashwinkumar/Downloads/imagenew.jpg")

for eachObject in detections:
    print(eachObject["name"] , " : " , eachObject["percentage_probability"] )

In [63]:
detections

[]

In [44]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
(rects, weights) = hog.detectMultiScale(j, winStride=(4, 4), padding=(8, 8), scale=1.05)

# draw the original bounding boxes
for (x, y, w, h) in rects:
    cv2.rectangle(copy, (x, y), (x + w, y + h), (0, 0, 255), 2)

cv2.imshow("copy", copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
weights

array([[1.58297323]])

In [46]:
rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
pick = non_max_suppression(rects, probs=None, overlapThresh=0.15)

# draw the final bounding boxes
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(copy, (xA, yA), (xB, yB), (0, 255, 0), 2)
    
cv2.imshow("copy", copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
os.getcwd()

'/home/ashwinkumar/Projects'

In [1]:
plt.imshow(i, cmap="gray")

In [208]:
# edge = cv2.Canny(gray, 30, 75)
# cv2.imshow("edge", edge)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, (1, 1))
# thresh = cv2.dilate(edge, (10, 10), 30)
# cv2.imshow("thresh", thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]
    
bboxes = [cv2.boundingRect(c) for c in contours]
_, sorted_contours = zip(*sorted(zip(bboxes, contours), key=lambda x: x[0][0]))

# sorted_contours = [i for i in sorted_contours if len(i) > 4]
copy = cv2.drawContours(image.copy(), sorted_contours, 0, (0, 0, 255), 2)
bbox = cv2.boundingRect(sorted_contours[0])

x, y, w, h = sorted_contours[0]
roi = image[y: y + h, x: x + w]
cv2.rectangle(copy, bbox, (0, 0, 255), 2)
cv2.imshow("copy", copy)
cv2.waitKey(0)
cv2.destroyAllWindows()